In [1]:
from random import choice
from string import ascii_uppercase
import math
import time
from swalign import swalign
from scipy.stats import beta
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import simps
from numpy import trapz
import random
import operator


# ****** functions ********
#Reads the DB files and puts the information of the file in a array of strings
def readfile(filename):
    temp = open(filename, 'r').read().split('\n')
    return temp
    
    
#returns a random string of specified length
#length: strign length
def randomword(length):
    return (''.join(choice('ACGT') for i in range(0, length)))

#retuns an array of random strings
#size: how many strings there will be in the array
#lakeMinLen: min sequence length
#lakeMaxLen: max sequence length
def lakeString(size, lakeMinLen, lakeMaxLen):     
    lake_water = []
    for i in range(0, size):
        random.seed()
        #generates a random sequence length
        y = random.randint(lakeMinLen, lakeMaxLen)
        
        _str = randomword(y)
        lake_water.append(_str)
    return lake_water

#returns one array of strings that represents database data set
#dbMinSeqLen: min string length
#dbMaxSeqLen: max string length
#dbSize: max number sequences in the array
def dbStrings(dbMinSeqLen, dbMaxSeqLen, dbSize):
    _array = []
    #generates how many different sequences there will be for the array
    random.seed()
    x = random.randint(2, dbSize)

    for i in range(0, x):
        #generates a random sequence size
        random.seed()
        y = random.randint(dbMinSeqLen, dbMaxSeqLen)
        _str = randomword(y)
        _array.append(_str)
    return _array

#corrupts the string if it is under the corruption rate
def stringCorruption(strg, corrupt_rate):
    _str = ''
    for k in strg:
        #get a random number between 0 and 1
        random.seed()
        floatr = random.random()
        if  floatr > corrupt_rate :
            _str = _str + k
        else:
            _str = _str + choice('ACGT')     
    return _str

#configures the controlled data
#parameters:
#k_v: known_virus
#k_b: known_bacterias
#nControlled: how many records will be controlled
#error: corruption limit
def dataController(k_v, k_b, error, nControlVir, nControlBact, lakeMinLen, lakeMaxLen):
    controlled_lake = []
    vLen = len(known_viruses)
    bLen = len(known_bacterias)
    
    #creates the specified amount of controlled lake sequences with viral pieces
    for j in range (0, nControlVir):
        #get a random virus
        vir = known_viruses[random.randint(1, vLen-1)]
        #get the length of the random virus
        virLen = len(vir)            
        count = 0
        #tries to find valid random substrings to be used
        while(True):
            num1 = random.randint(0, virLen)
            num2 = num1 + random.randint(lakeMinLen, lakeMaxLen)
            if num2 > virLen-1:
                continue
            else:
                break
        if num1 < num2:
            #controls the max length
            if(num2 - num1) > lakeMaxLen:
                num2 = num1 + lakeMaxLen
                
            #get a substring
            strg = vir[num1:num2]
            #corrupts the string if it's under the error rate
            strg = stringCorruption(strg, error)
            controlled_lake.append(strg)
        elif num2 < num1:
            #controls the max length
            if(num1 - num2) > lakeMaxLen:
                num1 = num2 + lakeMaxLen
                
            #get a substring
            strg = vir[num2:num1]
            #corrupts the string if it's under the error rate
            strg = stringCorruption(strg, error)
            controlled_lake.append(strg)
        else:
            #controls the max length
            if(virLen - num2) > lakeMaxLen:
                num2 += lakeMaxLen
            else:
                num2 = virLen-1
                
            #get a substring
            strg = vir[num1:num2]
            #corrupts the string if it's under the error rate
            strg = stringCorruption(strg, error)
            controlled_lake.append(strg)  
    
    #creates the specified amount of controlled lake sequences with bacterial pieces
    for j in range (0, nControlBact):
        bact = known_bacterias[random.randint(1, bLen-1)]
        bactLen = len(bact)            
        count = 0
        while(True):
            num1 = random.randint(0, bactLen)
            num2 = num1 + random.randint(lakeMinLen, lakeMaxLen)
            if num2 > bactLen-1:
                continue
            else:
                break
        if num1 < num2:
            #controls the max length
            if(num2 - num1) > lakeMaxLen:
                num2 = num1 + lakeMaxLen
                
            #get a substring
            strg = bact[num1:num2]
            #corrupts the string if it's under the error rate
            strg = stringCorruption(strg, error)
            controlled_lake.append(strg)
        elif num2 < num1:
            #controls the max length
            if(num1 - num2) > lakeMaxLen:
                num1 = num2 + lakeMaxLen
                
            #get a substring
            strg = bact[num2:num1]
            #corrupts the string if it's under the error rate
            strg = stringCorruption(strg, error)
            controlled_lake.append(strg)
        else:
            #controls the max length
            if(virLen - num2) > lakeMaxLen:
                num2 += lakeMaxLen
            else:
                num2 = bactLen-1
                
            #get a substring
            strg = bact[num1:num2]
            #corrupts the string if it's under the error rate
            strg = stringCorruption(strg, error)
            controlled_lake.append(strg)
    
    return controlled_lake
    
# ******************************************************* main ****************************************************

pl = np.arange(20, 620, 20)
pl2 = []
erate = []

for n in pl:

    ####---------------- config. variables --------------------------------####
    percentage = 1      #how many will have virus/bacteria pieces
    bacterialRate = 0.3 #how many of the controlled lake sequences will bacterial piece
    error = 0.0         #corruption rate
    distLimit = 0       #distance between matching sequences

    lakeSize = 10    #max number of lake sequences
    lakeMinLen =  n - 20   #min lake string length
    if lakeMinLen == 0:
        lakeMinLen = 4
    lakeMaxLen = n     #max lake string length

    dbSize = 15  #max number sequences in the array
    dbMaxSeqLen = 600  #max db string length
    dbMinSeqLen = lakeMinLen #min db string length

    #print configuration
    print("\n\nConfig. Variables")
    print("  Controlled Data:", percentage*100, "%")
    print("  Percentage of Bact. data:", bacterialRate*100, "%")
    print("  Corruption Rate:", error*100, "%")
    print("  Ditance Limit:", distLimit)
    print("  Db string min lenght:", dbMinSeqLen)
    print("  Db string max lenght:", dbMaxSeqLen)
    print("  Db max size:", dbSize)
    print("  Lake string min lenght:", lakeMinLen)
    print("  Lake string max lenght:", lakeMaxLen)
    print("  Lake data max size:", lakeSize, "\n\n")

    #generates how many different sequences there will be for the lake
    random.seed()
    size = random.randint(2, lakeSize)

    #generates the database representation
    known_viruses = dbStrings(dbMinSeqLen, dbMaxSeqLen, dbSize)
    known_bacterias = dbStrings(dbMinSeqLen, dbMaxSeqLen, dbSize)
    #known_viruses = readfile("../database/virus_db.txt")
    #known_bacterias = readfile("../database/bact_db.txt")

    #generates the lake sequences w/ the specified percentage of controlled data
    P = math.ceil(size * percentage)
    lake = lakeString(size - P, lakeMinLen, lakeMaxLen) #produce a part with totally random sequences  

    #calculates how many sequences will be viral and how many will be bacterial
    nControlBact = round(P * bacterialRate)
    if bacterialRate == 0.0:
       nControlVir = P
    else:
       nControlVir = round((nControlBact * (1 - bacterialRate)) / bacterialRate)

    print("# inserted viral pieces:", nControlVir, "\n# inserted bacterial pieces: ", nControlBact)
    size = size - P + nControlVir + nControlBact
    truebact = nControlBact/size
    print("True bacterial percentage:", truebact*100, "%")

    #produce the other part with viral or bacterial pieces
    controlled = dataController(known_viruses, known_bacterias, error, nControlVir, nControlBact, lakeMinLen, lakeMaxLen)
    lake = lake + controlled #join the sequences
    #lake = readfile("../database/lake_data.txt")

    #testing sequences
    #print(lake)
    #print("\n")

    #print(known_bacterias)
    #print("\n")

    #print(known_viruses)
    #print("\n")

    #print sequences
    #i = 0
    #print("*** Lake Sequences")
    #for w in lake:
    #    print(i, " ", w)
    #    i = i + 1 
    #    
    #print("\n\n*** Viral Sequences")
    #m = 0
    #for w in known_viruses:
    #    print(m, " ", w)
    #    m = m + 1 
    #    
    #print("\n\n*** Baterial Sequences")
    #m = 0
    #for w in known_bacterias:
    #    print(m, " ", w)
    #    m = m + 1 
    #print("\n\n")

    nBact = 0    #final counter
    nVirus = 0   #final counter
    nUnknown = 0 #final counter
    _y = 1 #final curve
    _x = 0 #average

    #Local Alignment settings
    # choose your own values here… 2 and -1 are common.
    match = 1                                                  #scores 1 point for matching letters
    mismatch = -1                                              #looses 1 point for mismatching letters
    scoring = swalign.NucleotideScoringMatrix(match, mismatch)
    sw = swalign.LocalAlignment(scoring)                       # you can also choose gap penalties, etc...

    #draws beta distributions
    x = np.arange(0, 1, 0.001)

    start = time.clock() #start clock to measure proccessing time

    for l in lake:       #for each sequece in the lake
        #print("***l len: ", len(l))
        viralMatches = 0 
        bactMatches = 0
        lLen = len(l)                                  #get sequence length
        #gives different weights depending of the sequence length 
        pnts = math.ceil((lLen/dbMaxSeqLen*100) + 1)
        _limit = round(lLen * (100 - distLimit) / 100) #get the minimum points to be considered a match
        for v in known_viruses:                        #compare the sequence with all virus in the database
            alignment = sw.align(v, l)
            if alignment.score >= _limit:
                viralMatches += pnts       
        for b in known_bacterias: #compare the sequence with all bacterias in the database
            alignment = sw.align(b, l)
            if alignment.score >= _limit:
                bactMatches += pnts

        #draws beta distributions    
        bactMatches += 1
        viralMatches += 1    
        mean, var, skew, kurt = beta.stats(bactMatches, viralMatches, moments='mvsk')
        #_label = "Viral:", viralMatches, "Bacterial:", bactMatches
        #ax.plot(x, beta.pdf(x, bactMatches, viralMatches),'p-', lw=3, alpha=0.6, label=_label)
        y = beta.pdf(x, bactMatches, viralMatches)
        yIndex = np.argmax(_y)

        #multiplies the coordinates
        _y *= y 
        _x += lLen


    end = time.clock()
    print("\n\n>>>Elapsed Time:", round(end - start, 3))    

    #Gets the highest value, the peak value
    yIndex = np.argmax(_y)
    print("\npredicted: ", x[yIndex])
    erate.append(x[yIndex] - truebact)
    _x = round(_x/size)
    print("Average:", _x)
    pl2.append(_x)

    #print("\n\nReport")
    #print("   Viruses:", round((1 - x[yIndex]) * 100, 2), "%")
    #print("   Bacteria:", round(x[yIndex] * 100, 2), "%")

    #plots the multiplication between the beta distribution curves 
    fig, ax2 = plt.subplots(1, 1)
    #ax2.plot(x, _y, 'y-', lw = 5, alpha=0.9, label="label")
    ax2.set_title("Beta Distribution Multiplication")
    ax2.set_xlabel("Bacterial Rate")
    ax2.legend(loc='best', frameon=False)
   

print("\nErate: \n", erate)
print("\nlengths: \n", pl2)
plt.show()



Config. Variables
  Controlled Data: 100 %
  Percentage of Bact. data: 30.0 %
  Corruption Rate: 0.0 %
  Ditance Limit: 0
  Db string min lenght: 4
  Db string max lenght: 600
  Db max size: 15
  Lake string min lenght: 4
  Lake string max lenght: 20
  Lake data max size: 10 


# inserted viral pieces: 2 
# inserted bacterial pieces:  1
True bacterial percentage: 33.33333333333333 %


>>>Elapsed Time: 1.071

predicted:  0.286
Average: 8


Config. Variables
  Controlled Data: 100 %
  Percentage of Bact. data: 30.0 %
  Corruption Rate: 0.0 %
  Ditance Limit: 0
  Db string min lenght: 20
  Db string max lenght: 600
  Db max size: 15
  Lake string min lenght: 20
  Lake string max lenght: 40
  Lake data max size: 10 


# inserted viral pieces: 2 
# inserted bacterial pieces:  1
True bacterial percentage: 33.33333333333333 %


C:\Users\Geysa\Anaconda3\lib\site-packages\matplotlib\axes\_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "




>>>Elapsed Time: 4.253

predicted:  0.353
Average: 27


Config. Variables
  Controlled Data: 100 %
  Percentage of Bact. data: 30.0 %
  Corruption Rate: 0.0 %
  Ditance Limit: 0
  Db string min lenght: 40
  Db string max lenght: 600
  Db max size: 15
  Lake string min lenght: 40
  Lake string max lenght: 60
  Lake data max size: 10 


# inserted viral pieces: 2 
# inserted bacterial pieces:  1
True bacterial percentage: 33.33333333333333 %


>>>Elapsed Time: 10.004

predicted:  0.321
Average: 47


Config. Variables
  Controlled Data: 100 %
  Percentage of Bact. data: 30.0 %
  Corruption Rate: 0.0 %
  Ditance Limit: 0
  Db string min lenght: 60
  Db string max lenght: 600
  Db max size: 15
  Lake string min lenght: 60
  Lake string max lenght: 80
  Lake data max size: 10 


# inserted viral pieces: 5 
# inserted bacterial pieces:  2
True bacterial percentage: 28.57142857142857 %


>>>Elapsed Time: 21.442

predicted:  0.287
Average: 67


Config. Variables
  Controlled Data: 100 %
  Per

C:\Users\Geysa\Anaconda3\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)




Config. Variables
  Controlled Data: 100 %
  Percentage of Bact. data: 30.0 %
  Corruption Rate: 0.0 %
  Ditance Limit: 0
  Db string min lenght: 420
  Db string max lenght: 600
  Db max size: 15
  Lake string min lenght: 420
  Lake string max lenght: 440
  Lake data max size: 10 


# inserted viral pieces: 5 
# inserted bacterial pieces:  2
True bacterial percentage: 28.57142857142857 %


>>>Elapsed Time: 187.091

predicted:  0.286
Average: 427


Config. Variables
  Controlled Data: 100 %
  Percentage of Bact. data: 30.0 %
  Corruption Rate: 0.0 %
  Ditance Limit: 0
  Db string min lenght: 440
  Db string max lenght: 600
  Db max size: 15
  Lake string min lenght: 440
  Lake string max lenght: 460
  Lake data max size: 10 


# inserted viral pieces: 5 
# inserted bacterial pieces:  2
True bacterial percentage: 28.57142857142857 %


>>>Elapsed Time: 294.062

predicted:  0.283
Average: 453


Config. Variables
  Controlled Data: 100 %
  Percentage of Bact. data: 30.0 %
  Corruption Rat